In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '6'

In [2]:
import torch
import torchvision
import torchvision.transforms as T
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
import random
import numpy as np

In [ ]:
# def seed_everything(seed_number = s):
#   random.seed(seed_number)
#   np.random.seed(seed_number)
#   _ = torch.manual_seed(seed_number)
#   torch.cuda.manual_seed(seed_number)
#   torch.cuda.manual_seed_all(seed_number)
#   torch.backends.cudnn.deterministic = True
#   torch.backends.cudnn.benchmark = False
#   return seed_number

In [3]:
size = 224
mean  = (0.49139968, 0.48215827, 0.44653124)
std = (0.24703233, 0.24348505, 0.26158768)

# Defrine transforms

transforms = {
    'train' : T.Compose([
      T.RandomResizedCrop(size, scale = (0.5, 1.0)),
      T.RandomHorizontalFlip(),
      T.ColorJitter(brightness = 0.5, contrast = 0.5),
      T.RandomAffine(degrees = 20, translate = (0.2, 0.2), shear=(-20,20)),
      T.ToTensor(),
      T.Normalize(mean, std)
    ]),
    'test' : T.Compose([
        T.Resize(256),
        T.CenterCrop(size),
        T.ToTensor(),
        T.Normalize(mean, std)
    ])
}

In [4]:
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction

    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        focal_loss = self.alpha * (1 - pt) ** self.gamma * ce_loss

        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

In [ ]:
# import torch
# import torch.nn as nn
# import torch.nn.functional as F
# from torch.autograd import Variable

# class FocalLoss(nn.Module):
#     def __init__(self, gamma=2, size_average=False):
#         super(FocalLoss, self).__init__()
#         self.gamma = gamma
#         self.size_average = size_average

#     def forward(self, input, target):
#         if input.dim()>2:
#             input = input.view(input.size(0),input.size(1),-1)  # N,C,H,W => N,C,H*W
#             input = input.transpose(1,2)    # N,C,H*W => N,H*W,C
#             input = input.contiguous().view(-1,input.size(2))   # N,H*W,C => N*H*W,C
#         target = target.view(-1,1)

#         logpt = F.log_softmax(input)
#         logpt = logpt.gather(1,target)
#         logpt = logpt.view(-1)
#         pt = logpt.exp()

#         loss = -1 * (1-pt)**self.gamma * logpt
#         if self.size_average: return loss.mean()
#         else: return loss.sum()

In [ ]:
# resnet18 = models.resnet18(pretrained = True).to(device)
# in_features = resnet18.fc.in_features
# resnet18.fc = nn.Sequential(nn.Linear(in_features,10)).to(device)

In [5]:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')

In [6]:
from tqdm import tqdm
import time

In [7]:
def train():
  # num_epochs = 10
  # for epoch in range(num_epochs):
  model.train()
  running_loss = 0.0

  with tqdm(total=len(trainloader), desc=f'Epoch {epoch+1}/{total_epoch}', unit='batch') as progress_bar:
      for i, (inputs, labels) in enumerate(trainloader):
          inputs = inputs.to(device)
          labels = labels.to(device)

          optimizer.zero_grad()
          outputs = model(inputs)
          loss = criterion(outputs, labels)
          loss.backward()
          optimizer.step()

          running_loss += loss.item()
          progress_bar.set_postfix(loss=running_loss / (i + 1))
          progress_bar.update()

def test():
  model.eval()
  correct = 0
  total = 0

  with tqdm(total=len(testloader), desc='Testing', unit='batch') as progress_bar:
      for inputs, labels in testloader:
          inputs = inputs.to(device)
          labels = labels.to(device)

          outputs = model(inputs)
          _, predicted = torch.max(outputs.data, 1)
          total += labels.size(0)
          correct += (predicted == labels).sum().item()

          progress_bar.update()

  print(f'Accuracy about 10000 test images : accuracy = {100 * correct / total}')

In [8]:
import warnings
warnings.filterwarnings('ignore')

model_num = 2 # total number of models
total_epoch = 50 # total epoch
lr = 0.01 # initial learning rate

for s in range(model_num):
    # fix random seed
    seed_number = s
    random.seed(seed_number)
    np.random.seed(seed_number)
    _ = torch.manual_seed(seed_number)
    torch.cuda.manual_seed(seed_number)
    torch.cuda.manual_seed_all(seed_number)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    
    # Check if GPU is available
    


    # Load the CIFAR-10 dataset
    trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transforms['train'])
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=128, shuffle=True, num_workers=16)

    testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms['test'])
    testloader = torch.utils.data.DataLoader(testset, batch_size=128, shuffle=False, num_workers=16)

    # Define the ResNet-18 model with pre-trained weights
    resnet18 = models.resnet18(pretrained = True).to(device)
    in_features = resnet18.fc.in_features #int
    resnet18.fc = nn.Sequential(nn.Linear(in_features,10)).to(device)
    model = resnet18.to(device)

    # Define the loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9)
    # criterion = FocalLoss()
    # optimizer = optim.Adam(model.parameters(), lr=lr)
    # Define the learning rate scheduler
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

     
    # Train the model
    for epoch in range(total_epoch):
        train()
        test()
        scheduler.step()

    print('Finished Training')
    save_path = '/home/hyungyu/hwcifar10/ex1/resnet18_cifar10_%f_%d.pth' % (lr, seed_number)

    # 모델 가중치를 지정한 경로에 저장
    torch.save(model.state_dict(), save_path)

Files already downloaded and verified
Files already downloaded and verified


Testing: 100%|██████████| 79/79 [00:06<00:00, 11.42batch/s]


Accuracy about 10000 test images : accuracy = 87.06


Testing: 100%|██████████| 79/79 [00:06<00:00, 11.77batch/s]


Accuracy about 10000 test images : accuracy = 89.42


...

Accuracy about 10000 test images : accuracy = 96.51


Testing: 100%|██████████| 79/79 [00:06<00:00, 11.30batch/s]


Accuracy about 10000 test images : accuracy = 96.51


Testing: 100%|██████████| 79/79 [00:06<00:00, 11.59batch/s]


Accuracy about 10000 test images : accuracy = 96.64
Finished Training


In [8]:
size = 224
mean  = (0.49139968, 0.48215827, 0.44653124)
std = (0.24703233, 0.24348505, 0.26158768)

# Defrine transforms
transform_test = T.Compose([
    T.Resize(256),
    T.TenCrop(224),
    T.Lambda(lambda crops: torch.stack([T.Normalize((0.49139968, 0.48215827, 0.44653124), (0.24703233, 0.24348505, 0.26158768))(T.ToTensor()(crop)) for crop in crops]))

    # T.ToTensor(),
    # T.Normalize(mean, std)
])

In [9]:
testset2 = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform_test)
testloader2 = torch.utils.data.DataLoader(testset2, batch_size=128, shuffle=False, num_workers=16)

Files already downloaded and verified


In [10]:
models2 = []
model_num= 2
lr = 0.01

resnet18 = models.resnet18(pretrained = True).to(device)
in_features = resnet18.fc.in_features #int
resnet18.fc = nn.Sequential(nn.Linear(in_features,10)).to(device)
model = resnet18.to(device)

for i in range(model_num):
    # Define the ResNet-18 model with pre-trained weights
    model.load_state_dict(torch.load(f"/home/hyungyu/hwcifar10/ex1/resnet18_cifar10_%f_%d.pth" % (lr, i)))  # Load the trained weights
    model.eval()  # Set the model to evaluation mode
    model = model.to(device)  # Move the model to the GPU
    models2.append(model)

/home/hyungyu/anaconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/home/hyungyu/anaconda3/envs/torch/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [11]:
correct = 0
total = 0
with torch.no_grad():
  for data in testloader2:
    images, labels = data
    images, labels = images.to(device), labels.to(device)  # Move the input data to the GPU
    bs, ncrops, c, h, w = images.size()       
    outputs = torch.zeros(bs, 10).to(device)  # Initialize the output tensor with zeros
    for model in models2:
      model_output = model(images.view(-1, c, h, w))  # Reshape the input to (bs*10, c, h, w)
      model_output = model_output.view(bs, ncrops, -1).mean(1)  # Average the predictions of the 10 crops
      outputs += model_output
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum().item()

print('Accuracy of the ensemble on the 10000 test images: %f %%' % (100 * correct / total))

Accuracy of the ensemble on the 10000 test images: 96.950000 %
